# 第4回 その1: 単回帰分析
前回の講義で高い相関を示す例として紹介した気温とアイスクリーム売上のデータを使用して，気温からアイス売上を予測してみましょう。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/04"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/04/
# 現在のフォルダの中身を表示
%ls

`icecream.csv`というデータが表示されていることを確認してください。

## ステップ1: 単回帰分析
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`icecream.csv` を読み込み，データをプロットします。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('icecream.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

month = csv_data.loc[:, '年月']
# numpy用データ(ndarray型) として抽出する。
kion = csv_data.loc[:, '平均気温(℃)'].to_numpy()
ice = csv_data.loc[:, 'アイス支出額(円)'].to_numpy()

# 抽出したデータをプロット
plt.figure(figsize=(13,10))
x = np.arange(np.size(kion))
# 2分割した上段に気温をプロット
plt.subplot(2,1,1)
plt.plot(x, kion, color='b')
# 補助目盛線の描画
plt.grid(axis='x')
# x軸の目盛りラベルの設定
plt.xticks(x, month, rotation=90)
plt.ylabel('Temperature [degree]')
# 下段にアイス支出額をプロット
plt.subplot(2,1,2)
plt.plot(x, ice, color='b')
plt.grid(axis='x')
plt.xticks(x, month, rotation=90)
plt.ylabel('Expending for icecream [yen]')
plt.show()

単回帰分析の関数を以下に定義します。  
中身は第3回で紹介した直線近似を行う関数（03_02_approximate_function.ipynb の linear_approx 参照）とほぼ同じです。  
お互いの関数を見比べてみて，違いを確認してください。

In [ ]:
# 単回帰分析を行う
def slr(x, y):
  '''
      x: 説明変数
      y: 目的変数
  '''
  N = np.size(x)
  # 係数計算に必要な変数の計算
  xx = x * x
  xy = x * y
  
  # a と b の計算で共通する分母の計算
  bunbo = N * np.sum(xx) - np.sum(x)*np.sum(x)

  # a および b の計算
  a = (N*np.sum(xy) - np.sum(x)*np.sum(y)) / bunbo
  b = (np.sum(xx)*np.sum(y) - np.sum(xy)*np.sum(x)) / bunbo

  return (a, b)

上で定義した関数 `slr` を使って，説明変数を気温 `kion`，目的変数を `ice` として単回帰分析を行います。

In [ ]:
# 単回帰分析を実行し，回帰係数を求める。
(a, b) = slr(kion, ice)
print('regression function: ice = %f * kion + %f' % (a, b))

# 気温から推定されるアイスの売上
predicted = a * kion + b

気温から推定されたアイスの売上（Predicted value）と，実際のアイスの売上（True value）をプロットし，比較してみます。

In [ ]:
x = np.arange(np.size(kion))
plt.figure(figsize=(13,5))
plt.plot(x, ice, label='True value', color='b')
plt.plot(x, predicted, label='Predicted value', color='r')
plt.grid(axis='x')
plt.xticks(x, month, rotation=90)
plt.ylabel('Expending for icecream [yen]')
plt.legend()
plt.show()

推定値と実際のアイス売上の誤差を計算してみます。

In [ ]:
# 平均二乗誤差(mean square error)
mse = np.mean((ice - predicted)*(ice - predicted))
# 平均平方根二乗誤差 (root mean square error) = mse の平方根
rmse = np.sqrt(mse)

print('MSE = %f' % (mse))
print('RMSE = %f' % (rmse))

平均 121.4 円程度の誤差で推定できていることが分かりました。

横軸：気温，縦軸：アイス売上の散布図と，  
横軸：アイス売上推定値，縦軸：アイス売上実際値の散布図をそれぞれ表示してみましょう。  
  

In [ ]:
# 横軸：気温，縦軸：アイス売上として散布図をプロット
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.scatter(kion, ice, color='b')
plt.title('Temperature vs Expending for icecream')
plt.xlabel('Temperature [degree]')
plt.ylabel('Expending for icecream [yen]')

# 横軸：アイス売上の推定値，縦軸：アイス売上の実際の値として散布図をプロット
plt.subplot(1,2,2)
plt.scatter(predicted, ice, color='b')
plt.title('Predicted vs True Values of Expending for Icecream')
plt.xlabel('Predicted value [yen]')
plt.ylabel('True value [yen]')
plt.show()

二つの図を見比べると，軸の値自体は異なりますが，散布図の様子は同じに見えます。

次に，気温とアイス売上の相関係数と，アイス売上の推定値と実際値の相関係数を求めてみましょう。

In [ ]:
# 相関係数を求める関数
def calc_corr_coef(x, y):
  '''
      x, y: 相関を求めたいデータ
  '''
  mu_x = np.mean(x)
  mu_y = np.mean(y)
  std_x = np.std(x)
  std_y = np.std(y)

  corr_coef = np.mean((x - mu_x) * (y - mu_y)) / (std_x * std_y)

  return corr_coef

# 気温とアイス売上の相関係数
corr_coef = calc_corr_coef(kion, ice)
print('correlation coefficent between temperature and expending for icecream = %f' % (corr_coef))

# アイス売上の推定値と実際値の相関係数
corr_coef = calc_corr_coef(predicted, ice)
print('correlation coefficent between predicted vs true Values of expending for icecream = %f' % (corr_coef))

相関係数が全く同じ値になることが確認できました。

推定値と実際値が全く同じ場合＝推定が完璧にできている場合，相関係数は 1.0 になります。  
つまり，<font color="Red">**相関係数は推定がうまく行っているかの指標になっている**</font>と言えます。  

推定結果と実際値の間の相関係数が，説明変数(気温)と目的変数(アイス売上)の相関係数と一致するということは，単回帰分析による推定がうまく行くか否かは，実際に推定を行わなくても，説明変数と目的変数の相関係数を計算すれば分かるということを意味します。